In [1]:
#python example to train doc2vec model (with or without pre-trained word embeddings)

import gensim.models as g
import logging
import pandas as pd
import re

In [2]:
df = pd.read_csv("../pt_question_answers_updated.csv")

In [3]:
## remove html

CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

df["pt_answer"] = df["pt_answer"].apply(lambda x: cleanhtml(x))

df["pt_title"] = df["pt_title"].str.lower()
df["pt_answer"] = df["pt_answer"].str.lower()

In [4]:
df

,pt_post_id,pt_post_type_id,pt_accepted_answer_id,pt_creation_date,pt_score,pt_title,pt_body,pt_tags,pt_parent_id,context,pt_answer,question
0,34750268,1,34762233.0,2016-01-12T17:36:25.473,9,extracting the top-k value-indices from a 1-d ...,<p>Given a 1-D tensor in Torch (<code>torch.Te...,<python><lua><pytorch><torch>,NaN,<p>Just loop through the tensor and run your c...,as of pull request #496 torch now includes a b...,Extracting the top-k value-indices from a 1-D ...
1,38543850,1,38676842.0,2016-07-23T16:15:43.967,40,how to display custom images in tensorboard (e...,"<p>The <a href=""https://github.com/tensorflow/...",<python><tensorflow><matplotlib><pytorch><tens...,NaN,<p>It is quite easy to do if you have the imag...,it is quite easy to do if you have the image i...,How to Display Custom Images in Tensorboard (e...
2,41767005,1,43824857.0,2017-01-20T15:22:08.063,11,python wheels: cp27mu not supported,"<p>I'm trying to install pytorch (<a href=""htt...",<python><linux><unicode><pytorch>,NaN,<p>This is exactly that. \nRecompile python un...,this is exactly that. \nrecompile python under...,Python wheels: cp27mu not supported <p>I'm try...
3,41861354,1,54261158.0,2017-01-25T20:45:35.297,8,loading torch7 trained models (.t7) in pytorch,<p>I am using Torch7 library for implementing ...,<python><lua><pytorch><torch><pre-trained-model>,NaN,<p>The correct function is <code>load_lua</cod...,as of pytorch 1.0 torch.utils.serialization is...,Loading Torch7 trained models (.t7) in PyTorch...
4,41924453,1,42054194.0,2017-01-29T18:31:24.687,65,pytorch: how to use dataloaders for custom dat...,<p>How to make use of the <code>torch.utils.da...,<python><torch><pytorch>,NaN,"<p>Yes, that is possible. Just create the obje...","yes, that is possible. just create the objects...",PyTorch: How to use DataLoaders for custom Dat...
...,...,...,...,...,...,...,...,...,...,...,...,...
10758,74612146,1,NaN,2022-11-29T09:54:30.430,0,is it possible to perform quantization on dens...,<p>I have been trying to performing quantizati...,<machine-learning><pytorch><artificial-intelli...,NaN,<p>Here's how to do this on DenseNet169 from t...,here's how to do this on densenet169 from torc...,NaN
10759,74637151,1,NaN,2022-12-01T05:08:37.150,1,"why when the batch size increased, the epoch t...",<p>Epoch time means the time required to train...,<deep-learning><pytorch>,NaN,"<p>As you already noticed, there are many fact...","as you already noticed, there are many factors...",NaN
10760,74642594,1,NaN,2022-12-01T13:23:27.277,0,why does stablediffusionpipeline return black ...,"<p>I am using the <a href=""https://github.com/...",<python><pytorch><apple-m1><huggingface-transf...,NaN,<p>Apparently it is indeed an Apple Silicon (M...,apparently it is indeed an apple silicon (m1/m...,NaN
10761,74671399,1,NaN,2022-12-03T22:46:46.443,1,locating tags in a string in php (with respect...,<p>I want to create a function that labels the...,<php><string><pytorch><label><italics>,NaN,<p>I think I've got something. How about this:...,i think i've got something. how about this:\nf...,NaN


In [5]:
text = []
for idx, row in df.iterrows():
    string = 'question: ' + row['pt_title'] + '\n' + 'answer: ' + row['pt_answer']
    text.append(string)

In [6]:
from nltk.tokenize import word_tokenize

## convert text to gensim TaggedDocument

In [7]:
tagged_data = [g.doc2vec.TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(text)]

In [8]:
tagged_data[0]

TaggedDocument(words=['question', ':', 'extracting', 'the', 'top-k', 'value-indices', 'from', 'a', '1-d', 'tensor', 'answer', ':', 'as', 'of', 'pull', 'request', '#', '496', 'torch', 'now', 'includes', 'a', 'built-in', 'api', 'named', 'torch.topk', '.', 'example', ':', '&', 'gt', ';', 't', '=', 'torch.tensor', '{', '9', ',', '1', ',', '8', ',', '2', ',', '7', ',', '3', ',', '6', ',', '4', ',', '5', '}', '--', 'obtain', 'the', '3', 'smallest', 'elements', '&', 'gt', ';', 'res', '=', 't', ':', 'topk', '(', '3', ')', '&', 'gt', ';', 'print', '(', 'res', ')', '1', '2', '3', '[', 'torch.doubletensor', 'of', 'size', '3', ']', '--', 'you', 'can', 'also', 'get', 'the', 'indices', 'in', 'addition', '&', 'gt', ';', 'res', ',', 'ind', '=', 't', ':', 'topk', '(', '3', ')', '&', 'gt', ';', 'print', '(', 'ind', ')', '2', '4', '6', '[', 'torch.longtensor', 'of', 'size', '3', ']', '--', 'alternatively', 'you', 'can', 'obtain', 'the', 'k', 'largest', 'elements', 'as', 'follow', '--', '(', 'see', 'the',

In [9]:
#doc2vec parameters
vector_size = 300
window_size = 15
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
dm = 0 #0 = dbow; 1 = dmpv
worker_count = 1 #number of parallel processes


In [10]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [11]:
model = g.Doc2Vec(vector_size=300, window=window_size, min_count=1, sample=sampling_threshold, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1)

model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=200)

2023-03-07 15:49:51,450 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow+w,d300,n5,w15,s1e-05,t3>', 'datetime': '2023-03-07T15:49:51.450905', 'gensim': '4.3.0', 'python': '3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 15:55:03) \n[GCC 10.4.0]', 'platform': 'Linux-5.15.0-1026-aws-x86_64-with-glibc2.31', 'event': 'created'}
2023-03-07 15:49:51,452 : INFO : collecting all words and their counts
2023-03-07 15:49:51,452 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-03-07 15:49:51,813 : INFO : PROGRESS: at example #10000, processed 1952932 words (5416446 words/s), 69354 word types, 10000 tags
2023-03-07 15:49:51,895 : INFO : collected 73269 word types and 10763 unique tags from a corpus of 10763 examples and 2083448 words
2023-03-07 15:49:51,896 : INFO : Creating a fresh vocabulary
2023-03-07 15:49:52,141 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 73269 unique words (100.00% of original 73269, drops 0

In [12]:
model

In [13]:
model.save("model.bin")

# model = g.Doc2Vec.load("model.bin")

2023-03-07 16:13:43,903 : INFO : Doc2Vec lifecycle event {'fname_or_handle': 'model.bin', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-03-07T16:13:43.903505', 'gensim': '4.3.0', 'python': '3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 15:55:03) \n[GCC 10.4.0]', 'platform': 'Linux-5.15.0-1026-aws-x86_64-with-glibc2.31', 'event': 'saving'}
2023-03-07 16:13:43,904 : INFO : storing np array 'vectors' to model.bin.wv.vectors.npy
2023-03-07 16:13:43,988 : INFO : storing np array 'syn1neg' to model.bin.syn1neg.npy
2023-03-07 16:13:44,132 : INFO : not storing attribute cum_table
2023-03-07 16:13:44,206 : INFO : saved model.bin


In [ ]:
## getting nearest documents to query vector

In [14]:
top_10_questions = pd.read_csv("../top100questions.csv").iloc[:10].question.tolist()
top_10_questions

['How do I check if PyTorch is using the GPU?\n',
 'How do I save a trained model in PyTorch?\n',
 'What does .view() do in PyTorch?\n',
 'Why do we need to call zero_grad() in PyTorch?\n',
 'How do I print the model summary in PyTorch?\n',
 'How do I initialize weights in PyTorch?\n',
 'What does model.eval() do in pytorch?\n',
 "What's the difference between reshape and view in pytorch?\n",
 'What does model.train() do in PyTorch?\n',
 'What does .contiguous() do in PyTorch?\n']

In [15]:
d = top_10_questions[0]
test_data = word_tokenize(d.lower())

start_alpha=0.01
infer_epoch=100
vector = model.infer_vector(test_data)


In [16]:
sims = model.docvecs.most_similar([vector])
sims

/tmp/ipykernel_24319/216048301.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([vector])


[('10526', 0.7156686782836914),
 ('1196', 0.669828474521637),
 ('10434', 0.6680323481559753),
 ('7880', 0.6646056175231934),
 ('8621', 0.6634865999221802),
 ('8464', 0.6581156253814697),
 ('8044', 0.6433849930763245),
 ('1973', 0.6412198543548584),
 ('10729', 0.6395546793937683),
 ('9244', 0.6377626657485962)]

In [17]:
for i in sims:
    print(text[int(i[0])])
    print('***********************************')

question: how do i get my conda environment to recognize my gpu?
answer: check your cuda version. not all the version of cuda supports official pytorch. check the official website of pytorch

***********************************
question: is there a pytorch method to check the number of cpus?
answer: just use this :

os.cpu_count()


***********************************
question: automatically check available gpu on google colab
answer: there is no way to check what gpu is available. add the line:
!nvidia-smi

to the beginning of your code and then keep on disconnecting and reconnecting the runtime until you get the gpu that you want.

***********************************
question: a better way to make pytorch code agnostic to running on a cpu or gpu?
answer: how about

if torch.cuda.device_count() &gt; 1:
    model = torch.nn.dataparallel(model)
model = model.to(device)


?

you don't need dataparallel if you have only one gpu.

***********************************
question: how could i k

In [18]:
for i in top_10_questions:
    test_data = word_tokenize(i.lower())

    start_alpha=0.01
    infer_epoch=1000
    vector = model.infer_vector(test_data, alpha=start_alpha, epochs=infer_epoch)

    sims = model.docvecs.most_similar([vector])
    print('question: ',i)
    print(text[int(sims[0][0])])
    print('***********************************')

/tmp/ipykernel_24319/489009377.py:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([vector])


question:  How do I check if PyTorch is using the GPU?

question: how do i get my conda environment to recognize my gpu?
answer: check your cuda version. not all the version of cuda supports official pytorch. check the official website of pytorch

***********************************
question:  How do I save a trained model in PyTorch?

question: how to save a pytorch model?
answer: to save:
# save the weights of the model to a .pt file
torch.save(model.state_dict(), &quot;your_model_path.pt&quot;)

to load:
# load your model architecture/module
model = yourmodel()
# fill your architecture with the trained weights
model.load_state_dict(torch.load(&quot;your_model_path.pt&quot;))


***********************************
question:  What does .view() do in PyTorch?

question: pytorch, how can i make same size of tensor model(x) and answer(x)?
answer: you can use tensor.view

https://pytorch.org/docs/stable/tensors.html#torch.tensor.view

so something like

answer(x.data).view(-1, 1)


should 